In [ ]:
import urllib.request as req
import os.path, random
import json
import requests

key = '9bd94b7e582fdba45a601ed68f788300'
url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=9bd94b7e582fdba45a601ed68f788300&movieNm=쥬라기'

response = requests.get(url)

# 상태코드 200이면 정상 응답
if response.status_code == 200:
    data = response.json()  # JSON을 Python dict로 변환
    print(data)  # 전체 데이터 출력

    # 예: 영화 리스트 출력
    for movie in data["movieListResult"]["movieList"]:
        print(f"제목: {movie['movieNm']}, 장르: {movie['genreAlt']}")
else:
    print("요청 실패:", response.status_code)

# savename = "movieDB.json"

# # exists를 이용하여 경로에 해당하는 파일이 없으면
# # if문을 실행하여 url의 내용으로 파일 생성
# if not os.path.exists(savename):
#     req.urlretrieve(url, savename)

# # json 파일 분석
# items = json.load(open(savename, 'r', encoding='utf-8'))

# # s = oprn(savename, 'r', encoding*('utf-2).read()

{'movieListResult': {'totCnt': 39, 'source': '영화진흥위원회', 'movieList': [{'movieCd': '20252527', 'movieNm': '쥬라기 월드: 새로운 시작', 'movieNmEn': 'Jurassic World: Rebirth', 'prdtYear': '2025', 'openDt': '20250702', 'typeNm': '장편', 'prdtStatNm': '개봉', 'nationAlt': '미국', 'genreAlt': '어드벤처,액션,스릴러,공포(호러)', 'repNationNm': '미국', 'repGenreNm': '어드벤처', 'directors': [{'peopleNm': '가렛 에드워즈'}], 'companys': []}, {'movieCd': '2025A109', 'movieNm': '쥬라기 아일랜드', 'movieNmEn': 'Jurassic Park', 'prdtYear': '2024', 'openDt': '', 'typeNm': '장편', 'prdtStatNm': '기타', 'nationAlt': '중국', 'genreAlt': '액션', 'repNationNm': '중국', 'repGenreNm': '액션', 'directors': [], 'companys': []}, {'movieCd': '20242021', 'movieNm': '쥬라기캅스 극장판: 전설의 고대생물을 찾아라', 'movieNmEn': '', 'prdtYear': '2024', 'openDt': '20240815', 'typeNm': '장편', 'prdtStatNm': '개봉', 'nationAlt': '한국', 'genreAlt': '애니메이션', 'repNationNm': '한국', 'repGenreNm': '애니메이션', 'directors': [{'peopleNm': '이재민'}], 'companys': [{'companyCd': '20229242', 'companyNm': '(주)스튜디오버튼 '}]}, 

In [29]:
import requests
import json
from time import sleep
from datetime import datetime
import re

TMDB_API_KEY = '5349acaed8d7c3af060172341881019e'  # TMDb API 키
TMDB_BASE = "https://api.themoviedb.org/3"
MAX_MOVIES = 3000  # 최대 3000개로 제한
MOVIES_PER_PAGE = 20
MAX_PAGES = 1000  # 최대 페이지 수 (보통 500페이지면 10,000개 영화)

all_movies = []
today_str = datetime.now().strftime("%Y-%m-%d")

# 한글 포함 여부 체크 함수
def contains_hangul(text):
    if text is None:
        return False
    return bool(re.search(r'[가-힣]', text))

def get_director(movie_id):
    url = f"{TMDB_BASE}/movie/{movie_id}/credits"
    params = {"api_key": TMDB_API_KEY}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return ""
    data = res.json()
    directors = [c["name"] for c in data.get("crew", []) if c["job"] == "Director"]
    return directors[0] if directors else ""

for page in range(1, MAX_PAGES + 1):
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ko",
        "sort_by": "primary_release_date.desc",
        "include_adult": False,
        "page": page,
        "primary_release_date.lte": today_str
    }

    response = requests.get(f"{TMDB_BASE}/discover/movie", params=params)
    if response.status_code != 200:
        print(f"페이지 {page} 요청 실패: {response.status_code}")
        break

    data = response.json()
    movies = data.get("results", [])
    if not movies:
        print(f"페이지 {page}에 데이터 없음, 중단")
        break

    for m in movies:
        movie_id = m.get("id")
        title_ko = m.get("title", "")

        # 한글 제목 아니면 건너뜀
        if not movie_id or not contains_hangul(title_ko):
            continue

        director = get_director(movie_id)
        genre_ids = m.get("genre_ids", [])

        movie_info = {
            "movie_id": movie_id,
            "title_ko": title_ko,
            "title_en": m.get("original_title", ""),
            "overview": m.get("overview", ""),
            "genres": genre_ids,
            "director": director,
            "vote_average": m.get("vote_average", 0),
            "release_date": m.get("release_date", ""),
            "poster_path": m.get("poster_path", "")
        }
        all_movies.append(movie_info)

        if len(all_movies) >= MAX_MOVIES:
            print(f"최대 영화 수 {MAX_MOVIES}개 도달, 중단")
            break

    print(f"페이지 {page} 완료, 누적 영화 수: {len(all_movies)}편")

    if len(all_movies) >= MAX_MOVIES:
        break

    sleep(0.3)

def get_genre_mapping():
    url = f"{TMDB_BASE}/genre/movie/list"
    params = {"api_key": TMDB_API_KEY, "language": "ko"}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return {}
    data = res.json()
    return {g['id']: g['name'] for g in data.get('genres', [])}

genre_map = get_genre_mapping()
for m in all_movies:
    genre_ids = m.get("genres", [])
    m["genres"] = [genre_map.get(gid, "") for gid in genre_ids]

with open("movies_DB.json", "w", encoding="utf-8") as f:
    json.dump(all_movies, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료: 총 {len(all_movies)}편 → movies_DB.json")

페이지 1 완료, 누적 영화 수: 0편
페이지 2 완료, 누적 영화 수: 1편
페이지 3 완료, 누적 영화 수: 1편
페이지 4 완료, 누적 영화 수: 3편
페이지 5 완료, 누적 영화 수: 3편
페이지 6 완료, 누적 영화 수: 3편
페이지 7 완료, 누적 영화 수: 4편
페이지 8 완료, 누적 영화 수: 4편
페이지 9 완료, 누적 영화 수: 4편
페이지 10 완료, 누적 영화 수: 4편
페이지 11 완료, 누적 영화 수: 4편
페이지 12 완료, 누적 영화 수: 4편
페이지 13 완료, 누적 영화 수: 5편
페이지 14 완료, 누적 영화 수: 8편
페이지 15 완료, 누적 영화 수: 10편
페이지 16 완료, 누적 영화 수: 10편
페이지 17 완료, 누적 영화 수: 11편
페이지 18 완료, 누적 영화 수: 11편
페이지 19 완료, 누적 영화 수: 11편
페이지 20 완료, 누적 영화 수: 15편
페이지 21 완료, 누적 영화 수: 15편
페이지 22 완료, 누적 영화 수: 18편
페이지 23 완료, 누적 영화 수: 20편
페이지 24 완료, 누적 영화 수: 24편
페이지 25 완료, 누적 영화 수: 27편
페이지 26 완료, 누적 영화 수: 28편
페이지 27 완료, 누적 영화 수: 29편
페이지 28 완료, 누적 영화 수: 31편
페이지 29 완료, 누적 영화 수: 33편
페이지 30 완료, 누적 영화 수: 33편
페이지 31 완료, 누적 영화 수: 40편
페이지 32 완료, 누적 영화 수: 50편
페이지 33 완료, 누적 영화 수: 54편
페이지 34 완료, 누적 영화 수: 54편
페이지 35 완료, 누적 영화 수: 55편
페이지 36 완료, 누적 영화 수: 71편
페이지 37 완료, 누적 영화 수: 74편
페이지 38 완료, 누적 영화 수: 74편
페이지 39 완료, 누적 영화 수: 74편
페이지 40 완료, 누적 영화 수: 74편
페이지 41 완료, 누적 영화 수: 77편
페이지 42 완료, 누적 영화 수: 88편
페이지 43 완료, 누적 영

In [ ]:
# json파일에서 한글명이 있는 데이터로 다시 추출
import json
import re

# 한글 포함 여부 체크 함수
def contains_korean(text):
    if text is None:
        return False
    return bool(re.search('[가-힣]', text))

# 기존 JSON 파일 불러오기
with open("movies_DB.json", encoding="utf-8") as f:
    movies = json.load(f)

# 한글이 포함된 title_ko만 필터링
movies_with_korean_title = [movie for movie in movies if contains_korean(movie.get("title_ko", ""))]

print(f"한글이 포함된 제목의 영화 수: {len(movies_with_korean_title)}")

# 새 JSON 파일로 저장
with open("movies_with_korean_title.json", "w", encoding="utf-8") as f:
    json.dump(movies_with_korean_title, f, ensure_ascii=False, indent=2)

print("한글 제목이 포함된 영화 데이터만 저장 완료.")

한글이 포함된 제목의 영화 수: 540
한글 제목이 포함된 영화 데이터만 저장 완료.


In [ ]:
import urllib.request as req
import os.path, random
import json

from konlpy.tag import Okt


PATH = "C:/kosmo/public/" # PATH 경로 
file_name = "movies_with_korean_title.json"
POST_URL = "https://image.tmdb.org/t/p/w500/"
with open(PATH+file_name, encoding="utf-8") as f:
    movies = json.load(f)
# print(movies)

# for movie in movies:
#     print("영화 제목 : ", movie['title_ko'])
#     print("줄거리 : ", movie['overview'])
#     print(f"장르 : {[g for g in movie['genres']]}")
#     print(f"포스터 : {POST_URL+str(movie['poster_path'])}")


def get_tokens(title, overview):
    okt = Okt()
    words = okt.pos(title, stem=True, norm=True)
    token = []
    for word, pos in words:
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    words = okt.pos(overview, stem=True, norm=True)
    for word, pos in words:
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    return token
keyword = []
# get_tokens("난장판이 된 사건사고: 열기구 소년 집에서 만든 비행접시가 날아오르며, 여섯 살짜리 꼬마를 싣고 갔다? 이 황당한 사고에 전 미국이 혼란에 빠지는데. 실제로는 어떤 일이 있었던 것일까. 이 다큐멘터리를 통해 알아보자.")
for movie in movies:
    keyword.append(get_tokens(movie['title_ko'], movie['overview']))
for k in keyword:
    print(k)

    

입력하신 '쥬라기'과(와) 가장 비슷한 영화 제목은 '해바라기'입니다.

'쥬라기' 추천 영화 리스트:
매드 마스크: 유사도 0.996
인류존속의 미스터리 그리고 두 연인: 유사도 0.996
스위트홈 감독판: 유사도 0.996
개구리는 죽는다: 유사도 0.996
신명: 유사도 0.996


In [27]:
import urllib.request as req
import os.path, random
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec
import numpy as np

# --- 데이터 불러오기 및 경로 설정 ---
PATH = "C:/kosmo/public/"
file_name = "movies_with_korean_title.json"
POST_URL = "https://image.tmdb.org/t/p/w500/"
with open(PATH + file_name, encoding="utf-8") as f:
    movies = json.load(f)

# --- 형태소 토큰화 함수 (제목 + 개요) ---
okt = Okt()

def get_tokens(title, overview):
    title = title or ""
    overview = overview or ""
    token = []
    for word, pos in okt.pos(title, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    for word, pos in okt.pos(overview, stem=True, norm=True):
        if pos in ['Noun', 'Verb', 'Adjective']:
            token.append(word)
    return token

# --- 전체 영화 토큰 리스트 만들기 ---
keyword = []
for movie in movies:
    keyword.append(get_tokens(movie.get('title_ko'), movie.get('overview')))

# --- Word2Vec 모델 학습 ---
model = Word2Vec(keyword, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# --- 영화 토큰 리스트를 임베딩 벡터로 변환 ---
def get_movie_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# --- 전체 영화 벡터 리스트 생성 ---
movie_vectors = [get_movie_vector(tokens, model) for tokens in keyword]

# --- 코사인 유사도 함수 ---
def cosine_similarity(vec1, vec2):
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# --- 제목만 토큰화 (형태소 기반 유사 제목 찾기용) ---
def tokenize_title(text):
    if not text:
        return []
    return [word for word, pos in okt.pos(text, stem=True, norm=True) if pos in ['Noun', 'Verb', 'Adjective']]

# --- Jaccard 유사도 ---
def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

# --- 형태소 기반 유사 제목 찾기 ---
def find_similar_title_token_based(input_title, movies, threshold=0.1):
    input_tokens = set(tokenize_title(input_title))
    best_match = None
    best_score = 0.0
    for movie in movies:
        title = movie.get('title_ko')
        if not title:
            continue
        title_tokens = set(tokenize_title(title))
        score = jaccard_similarity(input_tokens, title_tokens)
        if score > best_score and score >= threshold:
            best_score = score
            best_match = title
    return best_match

# --- 최종 추천 함수 ---
def recommend_movie_token_based(input_title, movies, keyword, movie_vectors, model, top_n=5):
    matched_title = find_similar_title_token_based(input_title, movies)

    if matched_title is None:
        print(f"'{input_title}'과(와) 비슷한 영화 제목을 찾지 못했습니다. 전체 영화 평균으로 추천합니다.")
        input_vec = np.mean(movie_vectors, axis=0)
    else:
        print(f"입력하신 '{input_title}'과(와) 가장 비슷한 영화 제목은 '{matched_title}'입니다.")
        for m, tokens in zip(movies, keyword):
            if m.get('title_ko') == matched_title:
                input_vec = get_movie_vector(tokens, model)
                break

    similarity_scores = []
    for i, vec in enumerate(movie_vectors):
        sim = cosine_similarity(input_vec, vec)
        similarity_scores.append((i, sim))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    recommendations = []
    for idx, score in similarity_scores:
        if matched_title and movies[idx].get('title_ko') == matched_title:
            continue
        recommendations.append((movies[idx].get('title_ko'), score))
        if len(recommendations) >= top_n:
            break

    return recommendations

# --- 추천 테스트 ---
input_title = "쥬라기"
recs = recommend_movie_token_based(input_title, movies, keyword, movie_vectors, model)

print(f"\n'{input_title}' 추천 영화 리스트:")
for title, score in recs:
    print(f"{title}: 유사도 {score:.3f}")

입력하신 '쥬라기'과(와) 가장 비슷한 영화 제목은 '쥬라기 월드: 새로운 시작'입니다.

'쥬라기' 추천 영화 리스트:
심연의 공포: 유사도 0.996
단잠: 유사도 0.996
보간클로크: 유사도 0.996
아이스 로드: 벤전스: 유사도 0.996
안도: 유사도 0.996


In [ ]:
import requests
import json
from time import sleep
from datetime import datetime, timedelta

TMDB_API_KEY = '5349acaed8d7c3af060172341881019e'
TMDB_BASE = "https://api.themoviedb.org/3"
POSTER_URL = "https://image.tmdb.org/t/p/w500"
MAX_PAGES = 500  # TMDB 최대 페이지 제한

def get_director(movie_id):
    try:
        url = f"{TMDB_BASE}/movie/{movie_id}/credits"
        params = {"api_key": TMDB_API_KEY}
        res = requests.get(url, params=params)
        if res.status_code != 200:
            return ""
        data = res.json()
        directors = [c["name"] for c in data.get("crew", []) if c["job"] == "Director"]
        return directors[0] if directors else ""
    except:
        return ""

def get_genre_mapping():
    url = f"{TMDB_BASE}/genre/movie/list"
    params = {"api_key": TMDB_API_KEY, "language": "ko"}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return {}
    data = res.json()
    return {g['id']: g['name'] for g in data.get('genres', [])}

def fetch_movies_by_month(year, month, end_date_override=None):
    all_movies = []
    genre_map = get_genre_mapping()
    start_date = datetime(year, month, 1)
    if end_date_override:
        end_date = end_date_override
    else:
        if month == 12:
            end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
        else:
            end_date = datetime(year, month + 1, 1) - timedelta(days=1)

    page = 1
    while page <= MAX_PAGES:
        try:
            params = {
                "api_key": TMDB_API_KEY,
                "language": "ko",
                "sort_by": "primary_release_date.asc",
                "include_adult": False,
                "page": page,
                "primary_release_date.gte": start_date.strftime("%Y-%m-%d"),
                "primary_release_date.lte": end_date.strftime("%Y-%m-%d"),
            }
            res = requests.get(f"{TMDB_BASE}/discover/movie", params=params)
            if res.status_code != 200:
                print(f"❌ {year}-{month:02d} 페이지 {page} 요청 실패: {res.status_code}")
                break

            data = res.json()
            movies = data.get("results", [])
            if not movies:
                break

            for m in movies:
                title_ko = m.get("title", "")
                if not any('\uac00' <= ch <= '\ud7a3' for ch in title_ko):  # 한글 포함 체크
                    continue

                movie_id = m.get("id")
                director = get_director(movie_id)
                genre_ids = m.get("genre_ids", [])
                genres = [genre_map.get(gid, "") for gid in genre_ids]

                movie_info = {
                    "movie_id": movie_id,
                    "title_ko": title_ko,
                    "title_en": m.get("original_title", ""),
                    "overview": m.get("overview", ""),
                    "genres": genres,
                    "director": director,
                    "vote_average": m.get("vote_average", 0),
                    "release_date": m.get("release_date", ""),
                    "poster_path": POSTER_URL + m.get("poster_path") if m.get("poster_path") else "",
                }
                all_movies.append(movie_info)

            print(f"✅ {year}-{month:02d} 페이지 {page} 완료 (누적: {len(all_movies)}편)")
            page += 1
            sleep(0.5)
        except Exception as e:
            print(f"🚨 예외 발생: {e}")
            break

    return all_movies

def collect_movies(start_year, start_month, end_year, end_month):
    collected = {}
    y, m = start_year, start_month
    today = datetime.now()

    try:
        while (y < end_year) or (y == end_year and m <= end_month):
            print(f"\n📅 {y}-{m:02d} 수집 시작")
            if y == end_year and m == end_month:
                end_date_override = datetime(today.year, today.month, today.day)
            else:
                end_date_override = None

            movies = fetch_movies_by_month(y, m, end_date_override)
            for movie in movies:
                collected[movie["movie_id"]] = movie

            if m == 12:
                y += 1
                m = 1
            else:
                m += 1
    except Exception as e:
        print(f"⚠️ 중간 오류 발생: {e}")
    finally:
        filename = "movies_DB_partial.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(list(collected.values()), f, ensure_ascii=False, indent=2)
        print(f"\n📝 수집 데이터 {len(collected)}편 저장 완료 → {filename}")

    return list(collected.values())

# 🔽 실행: 2000년 1월 ~ 2025년 7월 15일까지 수집
movies_data = collect_movies(2000, 1, 2025, 7)